# Построение моделей

In [1]:
from sklearn.linear_model import LinearRegression as LR
import numpy as np
import pandas as pd
data = pd.read_csv('data_cleaned.csv')
data.head()

,Unnamed: 0,Bal,Brick,Dist,Floor,Kitsp,Livesp,Metrdist,New,Nfloor,Price,Rooms,Totsp,Walk
0,0,1,1,0.592859,4,3.135494,4.553877,7.0,0,8.0,71014225,3.0,4.770685,1.0
1,1,1,1,1.239630,6,3.621671,3.806662,9.0,0,14.0,85000000,3.0,4.737075,1.0
2,2,1,1,0.781791,7,2.564949,4.234107,6.0,0,8.0,55000000,3.0,4.820282,1.0
3,3,1,1,3.381916,2,2.079442,2.760010,20.0,1,4.0,2316750,1.0,3.430756,0.0
4,4,1,1,3.068028,2,2.468100,3.600048,15.0,1,19.0,3650000,2.0,4.301359,0.0


### Очень неплохо походили на линейные зависимости, где мы логарифмировали цену. В связи с разбросом цен хорошей метрикой качества для наших моделей могла бы стать MAPE или MSPE. Принимая всё это во внимание, будем предсказывать логарифм цен.

In [2]:
data = data.drop('Unnamed: 0', 1)
data.Price = data.Price.apply(np.log)
data.head()

,Bal,Brick,Dist,Floor,Kitsp,Livesp,Metrdist,New,Nfloor,Price,Rooms,Totsp,Walk
0,1,1,0.592859,4,3.135494,4.553877,7.0,0,8.0,18.078391,3.0,4.770685,1.0
1,1,1,1.239630,6,3.621671,3.806662,9.0,0,14.0,18.258162,3.0,4.737075,1.0
2,1,1,0.781791,7,2.564949,4.234107,6.0,0,8.0,17.822844,3.0,4.820282,1.0
3,1,1,3.381916,2,2.079442,2.760010,20.0,1,4.0,14.655676,1.0,3.430756,0.0
4,1,1,3.068028,2,2.468100,3.600048,15.0,1,19.0,15.110238,2.0,4.301359,0.0


### Начнём с классики - линейной регрессии.

In [29]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
lr = LR()
X_train, X_test, y_train, y_test = train_test_split(data.drop('Price', 1), data.Price, test_size = 0.33)
lr.fit(X_train, y_train)
print('MSE: ', mse(lr.predict(X_test), y_test))
print('MAE: ', mae(lr.predict(X_test), y_test))

MSE:  0.0409302979757
MAE:  0.14673349541


## MAE логарифма - 0.15, т.е. MAPE цены около 15%. Что ж, неплохо.

## Конечно же, попробуем xgboost.

In [8]:
from xgboost import XGBRegressor as XGB

In [31]:
%%time
xgb1 = XGB(min_child_weight=10, n_estimators=1000, max_depth=10, subsample=0.66)
xgb1.fit(X_train, y_train)
print('MSE: ', mse(xgb1.predict(X_test), y_test))
print('MAE: ', mae(xgb1.predict(X_test), y_test))

MSE:  0.02418303949
MAE:  0.109463382918
CPU times: user 19.5 s, sys: 160 ms, total: 19.7 s
Wall time: 5.24 s


## Эксперименты показывают, что сильно меньше MAE мы уже не сделаем. MAPE около 11%.

In [32]:
%%time
xgb1 = XGB(min_child_weight=7, n_estimators=10000, max_depth=20, subsample=0.66, colsample_bylevel=0.66)
xgb1.fit(X_train, y_train)
print('MSE: ', mse(xgb1.predict(X_test), y_test))
print('MAE: ', mae(xgb1.predict(X_test), y_test))

MSE:  0.0246662731064
MAE:  0.110538100007
CPU times: user 2min 1s, sys: 1.84 s, total: 2min 3s
Wall time: 35.8 s
